











Checklist
1. Make sure your "Regional Format" is set to "English (United Kingdom)" before Excel opens files that contain English date-like objects,
   if you want them to be parsed as English datetime. English (United States) regional format will convert them to American Datetime as possible.
   You have no way to prevent Excel from converting date-like strings or objects to datetime when opening a file. 

In [ ]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import time
import datetime
import re
import os
import glob
import math
import gc
import shutil

from config import config
import YK_assistant

In [ ]:
TRAIN_MODE = True

OPERATION = 'PRETRAIN'  # 'PRETRAIN'  'TRAIN_C'   'FINETUNE'   'TEST', 'BACKTEST'

MODEL_ID = 'YK.02'
HISTORY_LEN = 400
SEQ_YEAR_SPAN = 15
NOTE = 'TempTEST'

DATA_THEME = str(HISTORY_LEN).zfill(3) + '-' + str(SEQ_YEAR_SPAN).zfill(2) + '-' + NOTE
COPY_IDS_DS_DATA_FROM = None  # "_DATA_400-15-Test"  None

NUMBER_QUERIES = 3
STARTING_PERCENT = 0
ENDING_PERCENT = 0
TRAIN_PERCENT= 97   #------ 96
VALID_PERCENT = 3   #------ 4
TEST_PERCENT = 100 - TRAIN_PERCENT - VALID_PERCENT
FILTER_FOR_GOOD_DATA = True

BATCH_SIZE = 25 # 25 for headers 6.

TEAM_EMBS = 50  #
DECODE_BASE_DATE = False
EMBED_AB_COLS = False    # True: Pls choose a small LR so that we have plenty of train epochs and the embedded values have enough chance to seek their proper places.
ODDS_IN_ENCODER = False   #--------------
ODDS_IN_DECODER = True
ADDITIONAL_D_MODEL = 0   #------------ Try increase it when underfitting.

TRANSFORMER_LAYERS = 30        #----------30
TRANSFORMER_HEADS = 6
BALANCE_POS_CODE = True        # True: Weakens positional code compared to embedded values.
DROPOUT = 0.0  # 0.1
ADAPTORS_LAYERS = 0 #------------ 10
ADAPTORS_WIDTH_FACTOR = 30  # 30

# This is an exponential curve that hits STARTING_LEARNING_RATE at step zero and EXAMPLE_LEARNING_RATE at step EXAMPLE_LEARNING_STEP.
# lr(step) = STARTING_LEARNING_RATE * pow( pow(EXAMPLE_LEARNING_RATE/STARTING_LEARNING_RATE, 1/EXAMPLE_LEARNING_STEP), step )
STARTING_LEARNING_RATE = 5e-7
EXAMPLE_LEARNING_RATE = STARTING_LEARNING_RATE * 0.5    #------------ 0.5 worked.
EXAMPLE_LEARNING_STEP = 100

MODEL_ACTIVATION = 'softmax'    # 'softmax', 'sigmoid', 'relu', 'open'
LOSS_TYPE = 'profit'           # 'profit', 'entropy'
VECTOR_BETTING = False
STAKE_TECHNIQUE = 'independent'         # 'independent', 'majority', 'riskest'

MODEL_TYPE_CHECK = False
GET_VAL_LOSS_0 = False
IGNORE_HISTORY = False
SIMPLIFY_ADAPTOR = True

In [ ]:
COUNTRY = 'England'
FOCUS_BOOKIE = 1    # None
NUMBER_BOOKIES = 5  # Take William Hills, Bet&Win, and Bet365. Other bookies' odds list change over years and leagues.
PREFERED_ORDER = ['B365', 'WH']   # Implies the order only.
BOOKIE_TO_EXCLUDE = []    # 'BWIN' odds don't show up since mid Febrary 2025. This may reduce the effective NUMBER_BOOKIES.
DIVIISONS = ['E0', 'E1', 'E2', 'E3']    # 'EC', the Conference league, is excluded as some odds makers are not archived for the league since 2013.
REPEATING_BOOKIE = None    #-----------------------------------

"""
#-------------------- England ---------------------
Time range of data: 2004/2005 - 2024/2025
Leagues - Premiere, League 1, League 2, Championship, Conference (dropped out)
!!! Conference will be dropped out because they lack Shoot_cols and ShootT_cols since 2026. Those columns are important.
Bookies - Bookie1 : WH(William Hills), Bookie2: BW(Bet&Win), Bookie3 : B365(Bet365), Bookie4 : Mixed
William Hills, Bet365 on Premier: https://www.oddschecker.com/football/english/premier-league
William Hills, Bet365 on Championship: https://www.oddschecker.com/football/english/championship
William Hills, Bet365 on League-1: https://www.oddschecker.com/football/english/league-1
William Hills, Bet365 on League-2: https://www.oddschecker.com/football/english/league-2
William Hills, Bet365 on Conference: https://www.oddschecker.com/football/english/non-league/national-league
BWin(Bet&Win) on Premier and Chanpionship: https://sports.bwin.fr/fr/sports/football-4/paris-sportifs/angleterre-14
"""

# COUNTRY = 'Scotland'
# NUMBER_BOOKIES = 3  # Take ...

In [ ]:
def set_seed(seed): np.random.seed(seed); tf.random.set_seed(seed); random.seed(seed)
set_seed(23)    # For serendipity

In [ ]:
ODDS_COLS = []
for b in range(NUMBER_BOOKIES):
    ODDS_COLS += ['HDA'+str(b)+'H', 'HDA'+str(b)+'D', 'HDA'+str(b)+'A']

In [ ]:
id_cols = ['id']
Div_cols = ['Div']
Date_cols = ['Date']
Team_cols = ['HomeTeam', 'AwayTeam']
Odds_cols = ODDS_COLS
BB_cols = id_cols + Div_cols + Date_cols + Team_cols + Odds_cols

Half_Goal_cols = ['HTHG', 'HTAG']
Full_Goal_cols = ['FTHG', 'FTAG']
Goal_cols = Half_Goal_cols + Full_Goal_cols
Shoot_cols = ['HS', 'AS']
ShootT_cols = ['HST', 'AST']
Corner_cols = ['HC', 'AC']
Faul_cols = ['HF', 'AF']
Yellow_cols = ['HY', 'AY']    # H/A Yellow Cards, H/A Red Cards
AB_cols = Half_Goal_cols + Full_Goal_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols
Required_Non_Odds_cols = Div_cols + Date_cols + Team_cols + Half_Goal_cols + Full_Goal_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols

# underscore_prefixed lists have discontinued columns.
BBAB_cols = BB_cols + AB_cols

# Make sure Odds_cols comes first !!!
_Cols_to_Always_Normalize = Odds_cols

_Label_cols = Full_Goal_cols + Odds_cols

bbab_odds_start = BBAB_cols.index(Odds_cols[0])
bb_odds_start = BB_cols.index(Odds_cols[0])
label_odds_start = _Label_cols.index(Odds_cols[0])

In [ ]:
print(BBAB_cols)
print(BB_cols)
print(_Label_cols)

In [ ]:
history_folder = os.path.join('.', 'history')
country_folder = os.path.join('.', "data", "football-data-co-uk", COUNTRY)
gameHistory_folder_path = os.path.join(country_folder, 'Game_History')
if not os.path.exists(gameHistory_folder_path): os.makedirs(gameHistory_folder_path, exist_ok=False)
countryTheme_folder_path = os.path.join(country_folder, '_DATA_' + DATA_THEME)
if not os.path.exists(countryTheme_folder_path): os.makedirs(countryTheme_folder_path, exist_ok=False)
map_folder_path = os.path.join(countryTheme_folder_path, '_id_map')     # do not change to '_map' for backward compatibility
if not os.path.exists(map_folder_path): os.makedirs(map_folder_path, exist_ok=False)
dataset_foler_path = os.path.join(countryTheme_folder_path, '_dataaset')    # keep '_dataaset' as it for backward compatibility
if not os.path.exists(dataset_foler_path): os.makedirs(dataset_foler_path, exist_ok=False)
checkpoint_folder_path = os.path.join(country_folder, '_checkpoints')
if not os.path.exists(checkpoint_folder_path): os.makedirs(checkpoint_folder_path, exist_ok=False)
tokenizer_folder_path =  os.path.join(countryTheme_folder_path, '_tokenizer')
if not os.path.exists(tokenizer_folder_path): os.makedirs(tokenizer_folder_path, exist_ok=False)
map_filename_base = DATA_THEME
dataset_path = os.path.join(dataset_foler_path, map_filename_base + '-foundation-ds')

In [ ]:
#========================= CREATE_DADAFRAMES_v2 =============================

skip = False
YK_assistant.assign_seasonal_filenames(gameHistory_folder_path)
df_train, df_new = YK_assistant.CREATE_DADAFRAMES_v2(gameHistory_folder_path, countryTheme_folder_path, Required_Non_Odds_cols, NUMBER_BOOKIES, oddsGroupsToExclude = BOOKIE_TO_EXCLUDE, preferedOrder = PREFERED_ORDER, train_mode = TRAIN_MODE, skip=skip)
print("df_train: ", df_train.shape)
print("df_new: ", df_new.shape)

In [ ]:
tokenizer_team = YK_assistant.creat_team_tokenizer_uk(df_train, tokenizer_folder_path)
team_vocab = list(tokenizer_team.get_vocab().keys())

print(tokenizer_team.get_vocab_size())
print(tokenizer_team.get_vocab())
print(team_vocab)

teams = ['Tottenham', 'Arsenal', 'Liverpool', 'what?', 'Tottenham', 'Chelsea', 'e_t', 'Man United', '1234', '[HOME]', '[AWAY]']
teams = [team.strip() for team in [re.sub(r"\s", "_", item) for item in teams]]
teams = " ".join(teams)
encoding = tokenizer_team.encode(teams)
# encoding = tokenizer.encode("")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)
print(tokenizer_team.decode(encoding.ids))
teams, success = YK_assistant.convert_to_token(tokenizer_team, ['Manchester United', 'Man united', 'man_united', 'chelsea'])
print(teams)

similars = YK_assistant.find_most_similar_teams(team_vocab, 'manchester united', 3)
print(similars)

teams, candidates, success = YK_assistant.convert_to_token_with_candidates(tokenizer_team, ['mman United', 'Liberpool', 'chelsea'], candi_count=3)
print(candidates)


In [ ]:
if TRAIN_MODE and COPY_IDS_DS_DATA_FROM is not None:
    YK_assistant.remove_folder_contents(map_folder_path)
    source_map_folder_path = os.path.join(country_folder, COPY_IDS_DS_DATA_FROM, '_id_map')
    YK_assistant.copy_id_map(source_map_folder_path, map_folder_path, NOTE)

In [ ]:
#================= CREATE_MAP_v2 ========================

if TRAIN_MODE:
    targetLength = HISTORY_LEN

    df_sequence = df_train; df_base = df_train
    train_map = YK_assistant.CREATE_MAP_v2(map_folder_path, map_filename_base, targetLength, df_sequence, df_base, year_span=SEQ_YEAR_SPAN, testcount=-1)

    print("\nlen = ", len(train_map))

    seq_types = [sType for (sType, _) in train_map.values()]
    seq_types = list(set([(seq_types.count(sType), sType) for sType in seq_types]))
    seq_types.sort()
    print(seq_types)    

In [ ]:
# lengths = [len(ids) for (report, ids) in train_map.values()]
# maxLen = max(lengths)
# plt.hist(lengths, np.linspace(0, int(maxLen*1.1), int(maxLen*1.1) + 1))
# plt.ylim(plt.ylim())
# maxLen = max(lengths)
# # plt.plot([maxLen, maxLen], plt.ylim())
# plt.title(f'Max length of ids: {maxLen}')

# lengths = [len(ids) for (report, ids) in train_map.values()]
# maxLen = max(lengths)
# plt.hist(lengths, np.linspace(0, int(maxLen*0.9), int(maxLen*0.9) + 1))
# plt.ylim(plt.ylim())
# maxLen = max(lengths)
# # plt.plot([maxLen, maxLen], plt.ylim())
# plt.title(f'Max length of ids: {maxLen}')
# assert maxLen == HISTORY_LEN

MAX_TOKENS = HISTORY_LEN

In [ ]:
def standardize_raw_bbab(bbab, tokenizer_team, normalization_parms):

    label = []
    #---------------------- label, before changing bbab. They are raw full-time goals and raw odds.
    for col in _Label_cols:
        start = BBAB_cols.index(col)
        label.append(bbab[start])

    #----------------------- 
    start = BBAB_cols.index(Div_cols[0])
    Div = bbab[start]
    bbab[start] = DIVIISONS.index(Div)  # Assumes no n/a

    start = BBAB_cols.index(Team_cols[0]); end = BBAB_cols.index(Team_cols[-1]) + 1
    pair_str = [str(team) for team in bbab[start : end]]    # Team names are already normalized, removing/striping spaces.
    pair_text = " ".join(pair_str)
    pair_tokens = tokenizer_team.encode(pair_text).ids
    bbab[start : end] = pair_tokens # 0 for Unknown, by tokenizer trainig.

    #--------------------- normalize
    for col in _Cols_to_Always_Normalize:   # Odds_cols only now.
        (mean, std, maximum) = normalization_parms[col]
        start = BBAB_cols.index(col)
        bbab[start] = (bbab[start] - mean) / std

    #--------------------- columns for positional embedding
    start = BBAB_cols.index(Date_cols[0])   #
    date = bbab[start]
    bbab[start] = (datetime.datetime.combine(date, datetime.time(0,0,0)) - config['baseDate']).days  # either positive or negative

    #---------------------- bb only
    start = BBAB_cols.index(BB_cols[0]); end = start + len(BB_cols)     # 
    bb = bbab[start : end]

    return bbab, bb, label, date

def getDateDetails(date):
    baseYear = config['baseDate'].year
    date_details = tf.Variable([date.year - baseYear, date.month, date.day, date.weekday()], dtype=tf.int32, trainable=False)
    return date_details     # (4,)

filler = tf.zeros_like([0] * len(BBAB_cols), dtype=tf.float32)

def get_data_record(df_sequence, df_base, baseId, ids, tokenizer_team, normalization_parms):
    # try:
        # base_bbab = list(df_train.loc[df_train['id'] == baseId, BBAB_cols])
        base_bbab = list(df_base[df_base['id'] == baseId][BBAB_cols].iloc[0, :])  # base_bbab follows BBAB. list------
        base_bbab, base_bb, base_label, base_date = standardize_raw_bbab(base_bbab, tokenizer_team, normalization_parms)
        # base_bbab, base_bb, base_label, base_date
        baseId = tf.Variable(baseId, dtype=tf.int32, trainable=False)
        base_bbab = tf.Variable(base_bbab, dtype=tf.float32, trainable=False)    # (len(BBAB_cols),)
        base_bb = tf.Variable(base_bb, dtype=tf.float32, trainable=False)        # (len(BB_cols),)
        base_label = tf.Variable(base_label, dtype=tf.float32, trainable=False)  # (len(_Label_cols),)
        # print('3', base_bbab)
        # Default sequence.
        sequence = tf.transpose(tf.Variable([[]] * len(BBAB_cols), dtype=tf.float32, trainable=False))   # (0, len(BBAB_cols))
        # sequence = np.array([[]] * len(BBAB_cols), dtype=config['np_float']).T
        # print('3.5', sequence)
        baseDateDetails = getDateDetails(base_date) # (4,)

        concat = []
        for id in ids:
            bbab = list(df_sequence[df_sequence['id'] == id][BBAB_cols].iloc[0, :])   # bbab follows BBAB. list
            bbab, _, _, _ = standardize_raw_bbab(bbab, tokenizer_team, normalization_parms)   # bbab follows BBAB. list
            # check_normalization(bbab, normalization_parms)

            bbab = tf.Variable(bbab, dtype=tf.float32, trainable=False)[tf.newaxis, :]       # (1, len(BBAB_cols))
            # _bbab = bbab[0].numpy()
            # check_normalization(_bbab, normalization_parms)

            concat.append(bbab)     # concat doesn't create a new axis.

        if len(concat) > 0:
            sequence = tf.concat(concat, axis=0)    # (nSequence, len(BBAB_cols))
            # if sequence.shape[0] > 0:
            #     bbab = sequence[0].numpy()
            #     check_normalization(bbab, normalization_parms)

        seq_len_org = sequence.shape[0]
        nMissings = MAX_TOKENS - seq_len_org
        if nMissings > 0:
            patch = tf.stack([filler] * nMissings, axis=0)
            sequence = tf.concat([sequence, patch], axis=0)     # concat doesn't create a new axis. (MAX_TOKENS, len(BBAB_cols))
        base_bb = base_bb[tf.newaxis, :]    # shape: (seq_len = 1, len(BBAB_cols))
        baseDateDetails = baseDateDetails[tf.newaxis, :]
        mask = tf.Variable([1] * seq_len_org + [0] * nMissings, dtype=tf.int32, trainable=False) # (MAX_TOKENS,) ## DO NOT USE tf.constant !!! unstable.
        mask = mask[:, tf.newaxis] & mask[tf.newaxis, :]    # (MAX_TOKENS, MAX_TOKENS)

        return (baseId, sequence, base_bb, base_label, baseDateDetails, mask)


def CREATE_DATASET(df_sequence, df_base, existing_ds, total_map, tokenizer_team, normalization_parms):
    def generator():
        total_baseIds = [int(id) for id in list(total_map.keys())]

        print('total_baseIds', total_baseIds)

        max_id = -1; count = 0
        if existing_ds is not None : # and len(existing_ds) > 0:
            for z in existing_ds:
                (baseId, _, _, _, _, _, _) = z
                baseId = baseId.numpy()
                if baseId in total_baseIds:
                    if max_id < baseId: max_id = baseId
                    print("count: {}, existing baseId: {}".format(count, baseId), end='\r'); count += 1
                    yield z
        new_map = { id : value for (id, value) in total_map.items() if max_id < int(id)}

        for baseId, (seq_type, ids) in new_map.items():
            baseId = int(baseId)
            baseId, sequence, base_bb, base_label, baseDateDetails, mask = get_data_record(df_sequence, df_base, baseId, ids, tokenizer_team, normalization_parms)
            print("count: {}, new baseId: {}".format(count, baseId), end='\r'); count += 1
            # if count > 200: break
            yield (baseId, seq_type, sequence, base_bb, base_label, baseDateDetails, mask)

    ds = tf.data.Dataset.from_generator(
        generator,
        output_types=(tf.int32, tf.int32, tf.float32, tf.float32, tf.float32, tf.int32, tf.int32),
        output_shapes=(tf.TensorShape(()), tf.TensorShape(()), tf.TensorShape((MAX_TOKENS, len(BBAB_cols))), tf.TensorShape((1, len(BB_cols),)), tf.TensorShape((len(_Label_cols),)), tf.TensorShape((1, 4)), tf.TensorShape((MAX_TOKENS, MAX_TOKENS))),
        args=()
    )
    return ds

def repeat_odds(sequence, base_bb, base_label, repeating_bookie):
    # sequence: (batch, max_seq_len, bbab_cols)
    # base_bb:  (batch, 1, bb_cols)
    # base_label: (batch, _Label_cols)
    def repeat_odds3(tensor, oddsStart):
        seedStart = oddsStart + repeating_bookie * NUMBER_QUERIES
        seedEnd = seedStart + NUMBER_QUERIES
        oddsSeed = tensor[:, :, seedStart : seedEnd]
        odds = tf.concat([oddsSeed] * NUMBER_BOOKIES, axis=-1)
        tensor = tf.concat( [tensor[:, :, : oddsStart], odds, tensor[:, :, oddsStart + NUMBER_BOOKIES * NUMBER_QUERIES :] ], axis=-1)
        return tensor
    
    def repeat_odds2(tensor, oddsStart):
        seedStart = oddsStart + repeating_bookie * NUMBER_QUERIES
        seedEnd = seedStart + NUMBER_QUERIES
        oddsSeed = tensor[:, seedStart : seedEnd]
        odds = tf.concat([oddsSeed] * NUMBER_BOOKIES, axis=-1)
        tensor = tf.concat( [tensor[:, : oddsStart], odds, tensor[:, oddsStart + NUMBER_BOOKIES * NUMBER_QUERIES :] ], axis=-1)
        return tensor

    sequence = repeat_odds3(sequence, bbab_odds_start)
    base_bb = repeat_odds3(base_bb, bb_odds_start)
    base_label = repeat_odds2(base_label, label_odds_start)

    return sequence, base_bb, base_label

   
def X_versus_Y(baseId, seq_type, sequence, base_bb, base_label, baseDateDetails, mask):
    if REPEATING_BOOKIE is not None:
        sequence, base_bb, base_label = repeat_odds(sequence, base_bb, base_label, REPEATING_BOOKIE)
    return (baseId, sequence, base_bb, baseDateDetails, mask), (base_label, seq_type)

# I found, in PositionalEmbedding, batch size ranges between 3, 4, 6 and 8, while they should be 4 or 8, except margial rows. Check it.
train_batch_size = BATCH_SIZE
test_batch_size = BATCH_SIZE * 2

def apply_train_pipeline(ds):
    return (
        ds
        .shuffle(ds.cardinality(), reshuffle_each_iteration=True)       # reshuffle doesn't happen at python iteration(ds)
        .batch(train_batch_size)
        .map(X_versus_Y, tf.data.AUTOTUNE)
        .cache()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
        )

def apply_test_pipeline(ds):
    return (
        ds
        .batch(test_batch_size)
        .map(X_versus_Y, tf.data.AUTOTUNE)
        .cache()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
        )

In [ ]:
if COPY_IDS_DS_DATA_FROM is not None:
    YK_assistant.remove_folder_contents(dataset_foler_path)
    source_map_folder_path = os.path.join(country_folder, COPY_IDS_DS_DATA_FROM, '_dataaset')
    YK_assistant.copy_dataset(source_map_folder_path, dataset_foler_path, NOTE)

In [ ]:
std_path = os.path.join(dataset_foler_path, map_filename_base + '-normalization' + ".json")
normalization_parms = YK_assistant.get_normalization_params(df_train, cols = _Cols_to_Always_Normalize + AB_cols)
print(normalization_parms)
YK_assistant.SaveJsonData(normalization_parms, std_path)
normalization_parms = YK_assistant.LoadJsonData(std_path)

In [ ]:
#======================= CREATE_DATASET ===========================

skip = True
if TRAIN_MODE:
    if os.path.exists(dataset_path):
        existing_ds = tf.data.Dataset.load(dataset_path, compression='GZIP')
        if skip == True: total_ds = existing_ds
        else:
            for z in existing_ds: break      # Required. "for z in ds" in generator will behavor as if ds is empty, if we remove this line.
            print("with existing ds...", len(existing_ds))
            df_sequence = df_train; df_base = df_train; total_map = train_map
            total_ds = CREATE_DATASET(df_sequence, df_base, existing_ds, total_map, tokenizer_team, normalization_parms)    # defines, not builds, total_ds.
            existing_data_subfolders = YK_assistant.get_subfolders(dataset_path)
            tf.data.Dataset.save(total_ds, dataset_path, compression='GZIP')       # builds what total_ds defines.
            total_ds = tf.data.Dataset.load(dataset_path) #, compression='GZIP')   # assign dataset to total_ds.
            YK_assistant.remove_subfolders(dataset_path, existing_data_subfolders)
    else:
        print('without existing ds...')
        df_sequence = df_train; df_base = df_train; existing_ds = None; total_map = train_map
        total_ds = CREATE_DATASET(df_sequence, df_base, existing_ds, total_map, tokenizer_team, normalization_parms)    # defines, not builds, total_ds.
        tf.data.Dataset.save(total_ds, dataset_path, compression='GZIP')       # builds what total_ds defines.
        total_ds = tf.data.Dataset.load(dataset_path, compression='GZIP')      # assign dataset to total_ds.

    print("The length of total dataset: {}       ".format(len(total_ds)))

    # len(total_ds) doesn't work, after applying a filter.
    # if FILTER_FOR_GOOD_DATA: 
    #     total_ds = total_ds.filter(lambda baseId, seq_type, sequence, base_bb, base_label, baseDateDetails, mask: seq_type == 20)
    #     for z in total_ds: pass
    #     print("The length of total dataset after filtering: {}       ".format(len(total_ds)))

In [ ]:
if TRAIN_MODE:
      total_size = len(total_ds)

      # ds = ds.shuffle(ds.cardinality(), seed=523) # Assuming the data is time-independent. seed does NOT work.

      starting_size = int(STARTING_PERCENT/100 * total_size)
      starting_size = int(STARTING_PERCENT/100 * total_size)
      ending_size = int(ENDING_PERCENT/100 * total_size)
      take_size = total_size - starting_size - ending_size
      remaining_ds = total_ds.skip(starting_size)
      dataset = remaining_ds.take(take_size)          # starting | dataset | ending

      dataset_size = len(dataset)
      train_size = int(TRAIN_PERCENT/100 * dataset_size)
      valid_size = int(VALID_PERCENT/100 * dataset_size)
      test_size = dataset_size - train_size - valid_size      # len(dataset) = train_size + valid_size + tast_size        NO back_size
      train_ds = dataset.take(train_size)                    # dataset[: train_size]
      remaining_ds = dataset.skip(train_size - valid_size)    # dataset[train_size - valid_size: ]
      back_ds = remaining_ds.take(valid_size)                # dataset[train_size - valid_size: train_size]
      remaining_ds = remaining_ds.skip(valid_size)            # dataset[train_size: ]
      valid_ds = remaining_ds.take(valid_size)               # dataset[train_size, train_size + valid_size]
      remaining_ds = remaining_ds.skip(valid_size)                # dataset[train_size + valid_size :]
      test_ds = remaining_ds.take(test_size)
      backtest_ds = valid_ds.concatenate(test_ds) # tf.data.Dataset.zip((valid_ds, test_ds))

      assert len(test_ds) == test_size
      assert dataset_size == len(train_ds) + len(valid_ds) + len(test_ds)

      print("total_size, dataset, train_ds, back_ds, valid_ds, test_ds, backtest_ds: ", \
            total_size, len(dataset), len(train_ds), len(back_ds), len(valid_ds), len(test_ds), len(backtest_ds))

In [ ]:
config['baseDate'] = datetime.datetime(2000, 1, 1)
dt = config['baseDate']
dt.year
dt = datetime.datetime(config['baseDate'].year + 5, 3, 15)
dt
dt2 = datetime.datetime(config['baseDate'].year + 10, 3, 15)
td = dt2 - dt
td.days

In [ ]:

if TRAIN_MODE:
    train_batches = apply_train_pipeline(train_ds)
    back_batches = apply_test_pipeline(back_ds)
    valid_batches = apply_test_pipeline(valid_ds)
    test_batches = apply_test_pipeline(test_ds)
    backtest_batches = apply_test_pipeline(backtest_ds)

In [ ]:
if TRAIN_MODE and MODEL_TYPE_CHECK:
    cnt = 5
    for z in train_batches:
        (baseId, sequence, base_bb, baseDateDetails, mask), (base_label, seq_type) = z
        cnt -= 1 
        if cnt == 0: break
    print(baseId.shape, sequence.shape, base_bb.shape, baseDateDetails.shape, mask.shape, base_label.shape, seq_type.shape)
    sample_x = (sequence, base_bb, baseDateDetails, mask)
    sample_y = (base_label, seq_type)
    # print(baseId.numpy(), base_bb.numpy(), baseDateDetails.numpy()) 

In [ ]:
# class PositionalEmbedding(tf.keras.layers.Layer):
class PositionalEmbedding(tf.keras.Model):

    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # if BALANCE_POS_CODE:
        #     self.total_years = 10   # not sure if it will cover all (baseDayss - sequenceDays)
        # else:
        self.total_years = SEQ_YEAR_SPAN  # datetime.datetime.now().year - config['baseDate'].year + 1 + 3   # 3 extra years. ------------
        self.total_days = 365 * self.total_years

        positional_resolution = d_model
        quotient = self.total_days / positional_resolution
        positions = tf.constant(range(self.total_days), dtype=tf.float32)    # (total_days,). range [0, total_days)
        fractional_pos = positions / quotient  # (total_days,). range (0, d_model)
        half_depth = d_model/2   #
        depths = tf.range(half_depth, dtype=tf.float32) / half_depth  # (half_depth,). range [0, 1), linear.
        BIG = d_model * 0.8
        depths = 1.0 / tf.pow(BIG, depths)        # (depth,). range [1, 1/BIG)
        angle_rads = fractional_pos[:, tf.newaxis] * depths  # (total_days, half_depth,). range [dayPos, dayPos/BIG) for each day
        pos_encoding = tf.concat([tf.math.sin(angle_rads), tf.math.cos(angle_rads)], axis=-1)   # Seperated sin and cos. (batch, seq_len, d_model)
        self.total_positional_code = pos_encoding
        return

    def call(self, x, sequenceDays, baseDays, isEncoder):

        if IGNORE_HISTORY: pass # x: (batch, MAX_TOKENS or 1, hParams.d_model)
        else:
            if BALANCE_POS_CODE:
                # sequneceDays - (baseDays - totalDays),  baseDays - (baseDays - totalDays)
                positions = tf.cast(baseDays - sequenceDays, dtype=tf.int32) if isEncoder else tf.cast(baseDays - baseDays, dtype=tf.int32)
            else:
                positions = tf.cast(self.total_days - sequenceDays, dtype=tf.int32) if isEncoder else tf.cast(self.total_days - baseDays, dtype=tf.int32)
            positional_code = tf.gather(self.total_positional_code, positions)
            if BALANCE_POS_CODE:
                positional_code /= tf.math.sqrt(tf.cast(x.shape[-1], tf.float32)) ################# rather than multiply to x. This makes a balance. Not in "Attention is all you need."
            x = x + positional_code
        return x

In [ ]:
pe = PositionalEmbedding(150)
positions = tf.constant(range(pe.total_days), dtype=tf.int32)
positions = pe.total_days - positions
pcode = tf.gather(pe.total_positional_code, positions)
# print(pcode)
plt.pcolormesh(pcode.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

In [ ]:
pe = PositionalEmbedding(150)
positions = tf.constant(range(0, 365 * 25, 365), dtype=tf.int32)
positions = pe.total_days - positions
pcode = tf.gather(pe.total_positional_code, positions)
# print(pcode)
plt.pcolormesh(pcode.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

In [ ]:

class hParams:    
    nDivisions = len(DIVIISONS) 
    division_embs = 4   # [0, 1, 2, 3]

    nTeams = tokenizer_team.get_vocab_size()    # including Unknown
    team_embs = TEAM_EMBS
    nGoals  = 4  # [0, 1, 2, 3] Maximum nGoals goals for a team in each of 1st and 2nd halfs. Extra goals will be clipped_by_value.
    goal_embs = 4
    nShoots = 21    # [0, ..., 20]
    shoot_embs = 4 # for combination
    nShootTs = 11   # [0, ..., 10]
    shootT_embs = 4 # for combination
    nCorners = 11   # [0, ..., 10]
    corner_embs = 4 # for combination
    nFauls = 21     # [0, ..., 20]
    faul_embs = 2 # for combination
    nYellows = 5    # [0, ..., 4]
    yellow_embs = 2 # for combination
    
    batch_size = BATCH_SIZE
    days_spanning_years = 30
    num_layers = TRANSFORMER_LAYERS
    num_heads = TRANSFORMER_HEADS
    m365_size = 1
    initial_m365 = 0.9

    # d_model DEPENDS...
    # We want as small d_model as possible, because from which we can freely choose an actual d_model.
    # Tests seem to indicate that larger d_model leads to training overfitting and validation underfitting.
    
    # Where goes ODDS_IN_DECODER ????????????????????

    d_model = 1 * division_embs + 2 * team_embs     # 1 division, 2 teams
    if ODDS_IN_ENCODER: d_model = d_model + (NUMBER_BOOKIES if FOCUS_BOOKIE is None else 1) * NUMBER_QUERIES  # + NUMBER_QUERIES odds * nBookies
    d_encoder = d_decoder = 0
    if EMBED_AB_COLS:
        d_encoder = d_model + 1 * goal_embs + 1 * goal_embs + 1 * (shoot_embs + shootT_embs + corner_embs + faul_embs + yellow_embs)
        if DECODE_BASE_DATE: d_decoder = d_model + 2 * 4     # 2 * 4 : date details
        else: d_decoder = d_model
    else:   # This mode, EMBED_AB_COLS = False, gives much smaller d_moddel, maybe avoiding overfitting.
        d_encoder = d_model + 1 * len(AB_cols)
        if DECODE_BASE_DATE: d_decoder = d_model + 2 * 4     # 2 * 4 : date details
        else: d_decoder = d_model
    d_model = max(d_encoder, d_decoder)     # (136, 118) for EMBED_AB_COLS, (126, 118) for False EMBED_AB_COLS
    d_model += ADDITIONAL_D_MODEL               # Adjust for the model size and overfitting.
    d_model = d_model + d_model % 2     # make it an even number.
    print("d_model raw: ", d_model)
    d_model = int( (d_model-1) / num_heads) * num_heads + num_heads  # make it a multiple of num_heads, closest to raw d_model.
    print("d_model refined: ", d_model)

In [ ]:
class Preprocess(tf.keras.Model):
    def __init__(self, hParams, isEncoder):
        super().__init__()
        self.isEncoder = isEncoder

        # game
        self.division_emb = tf.keras.layers.Embedding(hParams.nDivisions, hParams.division_embs, dtype=tf.float32, mask_zero=False) # Learn Unknown
        self.team_emb = tf.keras.layers.Embedding(hParams.nTeams, hParams.team_embs, dtype=tf.float32, mask_zero=False) # Learn Unknown

        if self.isEncoder:
            if EMBED_AB_COLS:
                # AB_cols
                self.firstH_goal_emb = tf.keras.layers.Embedding(hParams.nGoals * hParams.nGoals, hParams.goal_embs, dtype=tf.float32, mask_zero=False)
                self.secondH_goal_emb = tf.keras.layers.Embedding(hParams.nGoals * hParams.nGoals, hParams.goal_embs, dtype=tf.float32, mask_zero=False)
                self.shoot_emb = tf.keras.layers.Embedding(hParams.nShoots * hParams.nShoots, hParams.shoot_embs, dtype=tf.float32, mask_zero=False)
                self.shootT_emb = tf.keras.layers.Embedding(hParams.nShootTs * hParams.nShootTs, hParams.shootT_embs, dtype=tf.float32, mask_zero=False)
                self.corner_emb = tf.keras.layers.Embedding(hParams.nCorners * hParams.nCorners, hParams.corner_embs, dtype=tf.float32, mask_zero=False)
                self.faul_emb = tf.keras.layers.Embedding(hParams.nFauls * hParams.nFauls, hParams.faul_embs, dtype=tf.float32, mask_zero=False)
                self.yellow_emb = tf.keras.layers.Embedding(hParams.nYellows * hParams.nYellows, hParams.yellow_embs, dtype=tf.float32, mask_zero=False)
            else:
                params = []
                for col in AB_cols:
                    params.append(normalization_parms[col])
                self.AB_mormalization_params = tf.Variable(params, dtype=tf.float32, trainable=False)  # (num AB_cols=14, 3 = <mean, std, maximum>)

        if not self.isEncoder:
            if DECODE_BASE_DATE:
                self.day_emb = tf.keras.layers.Embedding(31, 2, dtype=tf.float32, mask_zero=False)
                self.month_emb = tf.keras.layers.Embedding(12, 2, dtype=tf.float32, mask_zero=False)
                self.wday_emb = tf.keras.layers.Embedding(7, 2, dtype=tf.float32, mask_zero=False)

        if IGNORE_HISTORY: pass
        else: self.dimensional_permution = tf.keras.layers.Dense(hParams.d_model)

        self.idx_Days = BB_cols.index('Date')

    def representDateDetails(self, dateDetails):
        # dateDetails: (batch, 1, 4)
        bYears, bMonths, bDays, bWDays = tf.split(dateDetails, [1, 1, 1, 1], axis=-1)   # All should be of (batch, seq_len = 1, 1)
        bYears = tf.cast(bYears, dtype=tf.float32)  # (batch, seq_len = 1, 1)
        bDays = self.day_emb(bDays)[:, :, -1]       # (batch, seq_len = 1, embs = 2)
        bMonths = self.month_emb(bMonths)[:, :, -1] # (batch, seq_len = 1, embs = 2)
        bWDays = self.wday_emb(bWDays)[:, :, -1]    # (batch, seq_len = 1, embs = 2)
        # w = tf.Variable(np.math.pi / 25, dtype=tf.float32, trainable=False)    # 25 years are covered by pi or a half circle.
        w = np.math.pi / 25
        bYearsCos = tf.math.cos(bYears * w)
        bYearsSin = tf.math.sin(bYears * w)
        bYears = tf.concat([bYearsCos, bYearsSin], axis=-1)   # (batch, seq_len = 1, 1+1 = 2)
        return bYears, bMonths, bDays, bWDays

    def combined_embeddings_of_double_columns(self, emb_layer, columns, nValues):
        # Assume emb_layer = Embedding(nValues * nValues, embs, mask_zero=False)
        cols = tf.cast(columns, dtype=tf.int32)
        cols = tf.clip_by_value(cols, 0, nValues-1)
        combi = cols[:, :, 0] * nValues + cols[:, :, 1]   # (batch, seq_len, 1). [0, ..., nValues * nValues - 1]
        combi = emb_layer(combi)
        return combi    # (batch, seq_len, 1)

    def call(self, x):
        (sequence, base_bb, baseDateDetails, mask) = x # sob = sequence or base_bb
        sequenceDays = sequence[:, :, self.idx_Days]  # (batch, seq_len)
        baseDays = base_bb[:, :, self.idx_Days]   # (batch, 1)

        # sequence follows BBAB, whereas base_bb follows 
        
        # BB_cols = id_cols + Div_cols + Date_cols + Team_cols + Odds_cols
        # AB_cols = Goal_cols + Result_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols + Red_cols

        if self.isEncoder:
            # ramainder: Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols + Red_cols  --- total 12 fields.
            id, div, days, teams, odds, half_goals, full_goals, shoot, shootT, corner, faul, yellow\
            = tf.split(sequence, [len(id_cols), len(Div_cols), len(Date_cols), len(Team_cols), len(Odds_cols), len(Half_Goal_cols), len(Full_Goal_cols), \
                                  len(Shoot_cols), len(ShootT_cols), len(Corner_cols), len(Faul_cols), len(Yellow_cols)], axis=-1)
            # All shape of (batch, sequence, own_cols), all tf.flaot32
        else:
            id, div, days, teams, odds, remainder \
            = tf.split(base_bb, [len(id_cols), len(Div_cols), len(Date_cols), len(Team_cols), len(Odds_cols), -1], axis=-1)  
            # remainder: [] 
            # All shape of (batch, 1, own_cols), guess., all tf.float32

        if FOCUS_BOOKIE is not None:
            odds = odds[:, :, FOCUS_BOOKIE * NUMBER_QUERIES: (FOCUS_BOOKIE+1) * NUMBER_QUERIES]
        
        div = self.division_emb(tf.cast(div, dtype=tf.int32))   # (batch, MAX_TOKENS or 1, columns=1, division_embs)
        div = tf.reshape(div, [div.shape[0], div.shape[1], -1]) # (batch, MAX_TOKENS or 1, extended_columns=1*division_embs) --- 
        teams = self.team_emb(tf.cast(teams, dtype=tf.int32))   # (batch, MAX_TOKENS or 1, columns=2, team_embs)
        teams = tf.reshape(teams, [teams.shape[0], teams.shape[1], -1]) # (batch, MAX_TOKENS or 1, extended_columns=2*team_embs) --- 

        if self.isEncoder:
            if EMBED_AB_COLS:
                first_half_goals = self.combined_embeddings_of_double_columns(self.firstH_goal_emb, half_goals, hParams.nGoals)
                second_half_goals = self.combined_embeddings_of_double_columns(self.secondH_goal_emb, full_goals - half_goals, hParams.nGoals)
                shoot = self.combined_embeddings_of_double_columns(self.shoot_emb, shoot, hParams.nShoots)
                shootT = self.combined_embeddings_of_double_columns(self.shootT_emb, shootT, hParams.nShootTs)
                corner = self.combined_embeddings_of_double_columns(self.corner_emb, corner, hParams.nCorners)
                faul = self.combined_embeddings_of_double_columns(self.faul_emb, faul, hParams.nFauls)
                yellow = self.combined_embeddings_of_double_columns(self.yellow_emb, yellow, hParams.nYellows)
                if ODDS_IN_ENCODER: concat = [div, teams, odds, first_half_goals, second_half_goals, shoot, shootT, corner, faul, yellow]
                else: concat = [div, teams, first_half_goals, second_half_goals, shoot, shootT, corner, faul, yellow]

            else:   # normalize now
                # AB_cols = Half_Goal_cols + Full_Goal_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols
                AB_values = [half_goals, full_goals, shoot, shootT, corner, faul, yellow]   # all (batch, 2 cols)
                AB_values = tf.concat(AB_values, axis=-1) # (batch, seq_len, num_AB_cols=14)
                # self.AB_mormalization_params  # (num AB_cols=14, 3 = <mean, std, maximum>)
                AB_values = (AB_values - self.AB_mormalization_params[:, 0]) / self.AB_mormalization_params[:, 1]
                if ODDS_IN_ENCODER: concat = [div, teams, odds, AB_values]
                else: concat = [div, teams, AB_values]
        else:
            if DECODE_BASE_DATE:
                bYears, bMonths, bDays, bWDays = self.representDateDetails(baseDateDetails)
                if ODDS_IN_DECODER:  concat = [div, teams, odds, bYears, bMonths, bDays, bWDays]
                else: concat = [div, teams, bYears, bMonths, bDays, bWDays]
            else:
                if ODDS_IN_DECODER: concat = [div, teams, odds]
                else: concat = [div, teams]

        concat = tf.concat(concat, axis=-1)
        assert concat.shape[-1] <= hParams.d_model        

        if IGNORE_HISTORY: pass # concat: (batch, MAX_TOKENS or 1, hParams.d_model)
        else:
            concat = self.dimensional_permution(concat)  # (batch, MAX_TOKENS or 1, hParams.d_model)

        if self.isEncoder:  mask = mask     # (batch, MAX_TOKEN, MAX_TOKEN), although (batch, 1, MAX_TOKEN) will propagate.
        else:   mask = mask[:, 0:concat.shape[1], :]    # concat: (batch, 1, MAX_TOKEN)

        return concat, mask, sequenceDays, baseDays


In [ ]:
isEncoder = True
if TRAIN_MODE and MODEL_TYPE_CHECK:
    pre = Preprocess(hParams, isEncoder=isEncoder)
    x, mask, seqDays, bDays = pre(sample_x); pre.summary()
    pos = PositionalEmbedding(hParams.d_model)
    x = pos(x, seqDays, bDays, isEncoder=isEncoder); pos.summary()
    @tf.function
    def fun(input):
        x, mask, sequenceDays, baseDays = pre(input)
        x = pos(x, sequenceDays, baseDays, isEncoder=isEncoder)
        return x, mask, sequenceDays, baseDays
    x, mask, sequenceDays, baseDays = fun(sample_x)
    print(x.shape, mask.shape, sequenceDays.shape, baseDays.shape)
    del pre, pos

In [ ]:
isEncoder = False
if TRAIN_MODE and MODEL_TYPE_CHECK:
    pre = Preprocess(hParams, isEncoder=isEncoder)
    x, mask, seqDays, bDays = pre(sample_x); pre.summary()
    pos = PositionalEmbedding(hParams.d_model)
    x = pos(x, seqDays, bDays, isEncoder=isEncoder); pos.summary()
    @tf.function
    def fun(input):
        x, mask, sequenceDays, baseDays = pre(input)
        x = pos(x, sequenceDays, baseDays, isEncoder=isEncoder)
        return x, mask, sequenceDays, baseDays
    x, mask, sequenceDays, baseDays = fun(sample_x)
    print(x.shape, mask.shape, sequenceDays.shape, baseDays.shape)
    del pre, pos

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
      # tf.keras.layers.Dropout(dropout_rate)
  ])

def scaled_dot_product_attention(q, k, v, mask=None):
  """
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v), seq_len_k == sel_len_v
    mask: Float 0/1 tensor with shape broadcastable to (..., seq_len_q, seq_len_k). 1 surpresses the score to zero.
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)  # (..., seq_len_q, seq_len_k)

  if mask is not None: scaled_attention_logits += (tf.cast(mask, dtype=tf.float32) * -1e9)

  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len_q, d_model)
    k = self.wk(k)  # (batch_size, seq_len_k, d_model)
    v = self.wv(v)  # (batch_size, seq_len_v, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, d_head)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, d_head)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, d_head)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, d_head)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, d_head)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights


class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):
    # value = x, key = x, query = x
    self_att, self_att_weights = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    self_att = self.dropout1(self_att, training=training)
    out1 = self.layernorm1(x + self_att)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out
  
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, context, training, 
           look_ahead_mask, padding_mask):
    # x: (batch, target_seq_len, d_model), context.shape: (batch_size, input_seq_len, d_model)

    # value = x, key = x, query = x
    self_att, self_att_weights = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len = 1, d_model)
    self_att = self.dropout1(self_att, training=training)
    self_att = self.layernorm1(self_att + x)

    # value = context, key = context, query = self_att
    cross_att, cross_att_weights = self.mha2(context, context, self_att, padding_mask)  # (batch_size, target_seq_len, d_model)
    cross_att = self.dropout2(cross_att, training=training)
    out2 = self.layernorm2(cross_att + self_att)  # (batch_size, target_seq_len, d_model)
    
    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
    
    return out, self_att_weights, cross_att_weights
  
class Encoder(tf.keras.layers.Layer):
    def __init__(self, hParams, dropout_rate=0.1):
      super().__init__()
      self.d_model = hParams.d_model
      self.num_layers = hParams.num_layers
      self.dropout = tf.keras.layers.Dropout(dropout_rate)
      self.enc_layers = [
          EncoderLayer(d_model=hParams.d_model, num_heads=hParams.num_heads, dff=hParams.d_model * 4, rate=dropout_rate)
          for _ in range(hParams.num_layers)]

    def call(self, x, training, encMask):
      x = self.dropout(x, training=training)
      for encoder_layer in self.enc_layers:
        x = encoder_layer(x, training, encMask)
      return x  # (batch_size, max_tokens, d_model)
 
class Decoder(tf.keras.layers.Layer):
    def __init__(self, hParams, dropout_rate=0.1):
      super(Decoder, self).__init__()
      self.d_model = hParams.d_model
      self.num_layers = hParams.num_layers
      self.dropout = tf.keras.layers.Dropout(dropout_rate)
      if IGNORE_HISTORY: pass
      else:
        self.dec_layers = [
            DecoderLayer(d_model=hParams.d_model, num_heads=hParams.num_heads, dff=hParams.d_model * 4, rate=dropout_rate)
            for _ in range(hParams.num_layers)]

    def call(self, x, context, training, look_ahead_mask, padding_mask):
      if IGNORE_HISTORY: pass
      else:
        x = self.dropout(x, training=training)
        for decoder_layer in self.dec_layers:
          x, _, _  = decoder_layer(x, context, training, look_ahead_mask, padding_mask)
      return x
  
class Transformer(tf.keras.Model):
    def __init__(self, hParams, dropout_rate=0.1):
      super().__init__()
      if IGNORE_HISTORY:
        self.all_one_context = tf.ones((BATCH_SIZE, MAX_TOKENS, hParams.d_model), dtype=tf.float32) # (batch, max_tokens, d_model)
      else:
        self.encPreprocess = Preprocess(hParams, isEncoder=True)
        self.decPreprocess = Preprocess(hParams, isEncoder=False)
        self.posEmbedding = PositionalEmbedding(hParams.d_model)
        self.encoder = Encoder(hParams, dropout_rate=dropout_rate)
        self.final_layer = tf.keras.layers.Dense(hParams.d_model) #-------------- to modify
      self.decoder = Decoder(hParams, dropout_rate=dropout_rate)
      self.one = tf.constant(1, dtype=tf.int32)

    def call(self, input, training=False):
      # inputs = (sequence, base_bb, baseDateDetails, mask)
      # sequence: (batch, max_token, aabb), base: (batch, 1, bb), baseDateDetails: (batch, 1, 4), mask: (batch, max_token, max_token)
      if IGNORE_HISTORY: 
        context = self.all_one_context
      else:
        x, encMask, sequenceDays, baseDays = self.encPreprocess(input) # (batch, max_tokens, d_model), (batch, max_tokens, max_tokens), (batch, max_tokens), (batch, 1)
        x = self.posEmbedding(x, sequenceDays, baseDays, isEncoder=True) # (batch, max_tokens, d_model)
        encMask = self.one - encMask    # NEGATE !!! forgoten for two YEARS !!!
        encMask = encMask[:, tf.newaxis, :, :]  # newaxis: head
        context = self.encoder(x, training, encMask)  # (batch, max_tokens, d_model). Only sequence and mask are used.

      x, decMask, sequenceDays, baseDays = self.decPreprocess(input) # (batch, 1, d_model), (batch, 1, max_tokens), (batch, max_tokens), (batch, 1)
      x = self.posEmbedding(x, sequenceDays, baseDays, isEncoder=False) # (batch, 1, d_model)
      decMask = decMask[:, tf.newaxis, :, :]
      # look_ahead_mask is None, which means [[0]], as there is only one position in x, so is nothing to mask when doing mha(value=x, key=x, query=x).
      x = self.decoder(x, context, training, look_ahead_mask=None, padding_mask=decMask)  # (batch, 1, d_model).  Only base_bb, baseDateDetails, and mask are used.      

      logits = self.final_layer(x)  # (batch, 1, d_model)
      logits = tf.squeeze(logits, axis=-2)  # (batch, d_model)

      return logits

In [ ]:
if TRAIN_MODE and MODEL_TYPE_CHECK:
    sample_transformer = Transformer(hParams, dropout_rate=DROPOUT)
    y = sample_transformer(sample_x, training=True)
    sample_transformer.summary()

    @tf.function
    def fun(x):
        y = sample_transformer(x, training=False)
        return y
    y = fun(sample_x)
    print(y.shape)
    del sample_transformer


In [ ]:
class Dense_Add_Norm(tf.keras.layers.Layer):
    def __init__(self, dim, seed):
      super().__init__()
      self.dense =tf.keras.layers.Dense (dim, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.5, seed=seed), bias_initializer=Zeros, activation='tanh')
      self.add = tf.keras.layers.Add()
      self.layernorm = tf.keras.layers.LayerNormalization()
    def call(self, x):
      dense = self.dense(x, training=False)
      x = self.add([x, dense])  # x.shape[-1] == dim
      x = self.layernorm(x)
      return x

Zeros = tf.keras.initializers.Zeros()

# Used for earlier versions that don't allow mixing bookies.
class Adaptor(tf.keras.Model):
  def __init__(self, nLayers, d_main, d_output, dropout_rate=0.1):
    super().__init__()
    # total (nLayers + nLayers) dims = 2 * nLayers dims
    dims = [d_main] * nLayers
    layers = [Dense_Add_Norm(dims[id], id) for id in range(len(dims))]
    self.seq = tf.keras.Sequential(layers)
    self.initial = tf.keras.layers.Dense (d_main, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.5, seed=23), activation='tanh')
    self.final = tf.keras.layers.Dense (d_output, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.5, seed=23), activation='tanh')
  def call(self, x, training=False):  # (batch, d_model)
    x = self.initial(x)
    x = self.seq(x)   # (batch, d_model)
    x = self.final(x) # (batch, nBookies * NUMBER_QUERIES)
    return x

In [ ]:
std_mean = tf.Variable([normalization_parms[col][0] for col in Odds_cols], trainable=False)
std_variation = tf.Variable([normalization_parms[col][1] for col in Odds_cols], trainable=False)
print(std_mean.shape, std_variation.shape)

# No, _Label_cols were not normalized.

In [ ]:
class Model_1X2(tf.keras.Model):
    softmax = tf.keras.layers.Softmax(axis=-1)

    def __init__(self, hParams, nQueries, dropout_rate=0.1):
        super().__init__()
        self.nQueries = nQueries
        self.transformer = Transformer(hParams, dropout_rate=dropout_rate)
        #   self.bookies = ['B365', 'Betfair', 'Interwetten', 'William']
        self.nBookies = NUMBER_BOOKIES if FOCUS_BOOKIE is None else 1

        if SIMPLIFY_ADAPTOR:
            self.adaptor = tf.keras.layers.Dense(self.nBookies * self.nQueries)
        else:
            self.adaptor = Adaptor(ADAPTORS_LAYERS, hParams.d_model * ADAPTORS_WIDTH_FACTOR, self.nBookies * self.nQueries)
     
        if LOSS_TYPE == 'entropy':
            self.categorical_crossentropy = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0.0, axis=-1, reduction='sum_over_batch_size')
        return

    def call(self, x, trainig=False):
        x = self.transformer(x, training=trainig) # (batch, d_model)
        output = self.adaptor(x)   # (batch, nBookies * nQueries)
        output = tf.reshape(output, [output.shape[0], self.nQueries, -1])    # (batch, nQueries, nBookies)
        output = tf.transpose(output, perm=[2, 0, 1])     # (nBookies, batch, nQueries)
        if MODEL_ACTIVATION == 'softmax':
            output = tf.nn.softmax(output)  # (nBookies, batch, nQueries)   #
        elif MODEL_ACTIVATION == 'sigmoid':
            output = tf.math.sigmoid(output * 5)  # the previous activation is tanh, ranging (-1, 1). Multiplier 5 will result in range (near 0, near 1)
        elif MODEL_ACTIVATION == 'relu':
            output = tf.nn.relu(output)
        elif MODEL_ACTIVATION == 'open':
            pass    # output = output
        return output   # (nBookies, batch, nQueries)
    
    def h_true(self, ftGoals):  # Defines this QGroup. This is for 1X2 QGroup. Derived classes re-define this funciton.
        # ftGoals:  (batch, 2)
        ftGoals = tf.cast(ftGoals, dtype=tf.int32)  # (batch, 2)
        h = (tf.math.greater(ftGoals[..., 0], ftGoals[..., 1]), tf.math.equal(ftGoals[..., 0], ftGoals[..., 1]), tf.math.less(ftGoals[..., 0], ftGoals[..., 1]))
        h = tf.cast(tf.transpose(h), dtype=tf.float32)  # (batch, nQueries)
        return h

    def loss(self, y, output):   
        # y = (base_label, seq_type):  ( (batch, len(Team_cols)+len(Odds_cols)), (batch,) ), 
        # output: (nBookies, batch, nQueries)
        (base_label, seq_type) = y

        if FILTER_FOR_GOOD_DATA: pass
            # apply a filter to both output and base_label, on the batch axis.

        def well(x, mu, sigma):
            norm = tf.norm(x)
            return tf.nn.relu(norm - sigma) * 0.5   # so this component of loss is less steep than the other component.
            # return - tf.math.exp(- tf.math.pow((x-mu)/sigma, 2) / 2) / (sigma * tf.math.sqrt(np.pi*2))

        ftGoals, odds = tf.split(base_label, [2, -1], axis=-1) # (batch, 2), (batch, len(self.bookies * self.nQueries))        if FOCUS_BOOKIE is not None:

        if FOCUS_BOOKIE is not None:
            odds = odds[:, FOCUS_BOOKIE * NUMBER_QUERIES: (FOCUS_BOOKIE+1) * NUMBER_QUERIES]

        odds = tf.split(odds, [self.nQueries] * self.nBookies, axis=-1)  # [(batch, nQueries)] * self.nBookies
        odds = tf.stack(odds, axis=0)  # (self.nBookies, batch, nQueries)
        happen_t = self.h_true(ftGoals) # (batch, nQueries)
        oh = tf.math.multiply(odds, happen_t)   # (self.nBookies, batch, nQueries)

        if LOSS_TYPE != 'entropy':
            (stake_p) = output  # (self.nBookies, batch, nQueries)
            # -----------------------------------------------------------------------------------------
            # Note: happen_p and stake_p are not converted to one-hot values, unlike they should.
            # Note: Do not normalize stake_p. It can learn whether to bet or not, as well as betting direction.
            #------------------------------------------------------------------------------------------
            profit_per_bookie_game = tf.reduce_sum(tf.math.multiply(oh - 1.0, stake_p), axis=-1)    # (nBooies, batch)
            mean_profit_by_game = tf.reduce_mean(profit_per_bookie_game, axis=0)    # (batch,)
            profit_backtest_sum = tf.reduce_sum(mean_profit_by_game, axis=None)  # () 
            loss_sum = - profit_backtest_sum  # U.action.42

            if MODEL_ACTIVATION == 'open':
                bell_loss = well(stake_p, 0, 2)
                loss += bell_loss

            if not VECTOR_BETTING:
                if STAKE_TECHNIQUE == 'independent':
                    stake_like = stake_p    # (self.nBookies, batch, nQueries)
                elif STAKE_TECHNIQUE == 'majority':
                    stake_like = tf.reduce_sum(stake_p, axis=0)     # (batch, nQueries)
                    stake_like = tf.stack([stake_like] * stake_p.shape[0], axis=0)  # (self.nBookies, batch, nQueries)
                elif STAKE_TECHNIQUE == 'riskest':
                    stake_like = tf.math.multiply(stake_p, odds)    # (self.nBookies, batch, nQueries)
                    stake_like = tf.reduce_sum(stake_like, axis=0)  # (batch, nQueries)
                    stake_like = tf.stack([stake_like] * stake_p.shape[0], axis=0)  # (self.nBookies, batch, nQueries)

                one_hot_stake_p = tf.squeeze(tf.one_hot(tf.nn.top_k(stake_like).indices, tf.shape(stake_like)[-1]), axis=-2)   # one_hot stake_p, (self.nBookies, batch, nQueries)
                profit_per_bookie_game = tf.reduce_sum(tf.math.multiply(oh - 1.0, one_hot_stake_p), axis=-1)    # (self.nBookies, batch)
                mean_profit_by_game = tf.reduce_mean(profit_per_bookie_game, axis=0)    # (batch,)
                profit_backtest_sum = tf.reduce_sum(mean_profit_by_game, axis=None)  # ()

        else:
            probability_p = tf.transpose(output, perm=[1, 2, 0])
            probability_p = tf.math.reduce_mean(probability_p, axis=-1) # (batch, nQueries)
            loss_sum = self.categorical_crossentropy(happen_t, probability_p)   # reduce sum over batch size. What over the axis -1.

            one_hot_stake_p = tf.squeeze(tf.one_hot(tf.nn.top_k(probability_p).indices, tf.shape(probability_p)[-1]), axis=1)   # one_hot stake_p
            
            profit_per_bookie_game = tf.reduce_sum(tf.math.multiply(oh - 1.0, one_hot_stake_p), axis=-1)    # (self.nBookies, batch)
            mean_profit_by_game = tf.reduce_mean(profit_per_bookie_game, axis=0)    # (batch,)
            profit_backtest_sum = tf.reduce_sum(mean_profit_by_game, axis=None)  # ()
    
        return loss_sum, profit_backtest_sum, mean_profit_by_game # (), (), (batch,)  Both loss_sum and profit_backtest_sum are a sum across batch. 


def create_model_object(model_class):
    try: model_c
    except NameError: pass
    else: del model_c
    try: model_1x2
    except NameError: pass
    else: del model_1x2
    tf.keras.backend.clear_session(); gc.collect()
    model = model_class(hParams, nQueries=NUMBER_QUERIES, dropout_rate=DROPOUT)   # Do not create a reference and return directly.
    return model

In [ ]:
model_1x2 = create_model_object(Model_1X2)

if TRAIN_MODE and MODEL_TYPE_CHECK:
    output = model_1x2(sample_x, training=True)
    @tf.function
    def fun(x, y):
        output = model_1x2(x, training=False)
        loss_sum, profit_sum, profits = model_1x2.loss(y, output)
        return output, loss_sum, profit_sum, profits
    output, loss_sum, profit_sum, profits = fun(sample_x, sample_y)
    print(output.shape, loss_sum.shape, profit_sum.shape, profits.shape)

    model_1x2.summary()
    # del model_1x2

In [ ]:
# tf.keras.utils.plot_model(model_1x2, show_shapes=True, dpi=48)

In [ ]:
class history_class():
    def round_sig(self, x, sig=2):
            return x
            # return round(x, sig-int(math.floor(math.log10(abs(x))))-1)    # domain error for VERY small numbers.
    def __init__(self, filepath):
        self.filepath = filepath
        self.history = {'initial_profit': -float('inf'), 'loss': [], 'val_loss_0': [], 'val_profit_0': [], 'val_loss': [], 'val_profit': [], 'learning_rate': [], 'recall': [], 'precision': [], 'time_taken': [],  'gauge': [], 'backtest_reports': []}
    def set_initial_interval_profit(self, initail_inverval_profit):
        self.history['initial_profit'] = initail_inverval_profit
        self.save()
    def removeFile(self):
        files = glob.glob(self.filepath + "*")   # "*.*" may not work
        result = [os.remove(file) for file in files]
    def save(self):
        YK_assistant.SaveJsonData(self.history, self.filepath)
    def reset(self):
        self.removeFile()
        # forgot to reset self.history? ---------------------- Check it.
        self.__init__(self.filepath)
        self.save()
    def load(self):
        history = YK_assistant.LoadJsonData(self.filepath)
        if history is not None: self.history = history

    def append(self, train_loss, val_loss_0, val_profit_0, val_loss, val_profit, learning_rate, recall, precision, time_taken, gauge, backtest_report):
        self.history['loss'].append(self.round_sig(train_loss, 4))
        self.history['val_loss_0'].append(self.round_sig(val_loss_0, 4))
        self.history['val_profit_0'].append(self.round_sig(val_profit_0, 4))
        self.history['val_loss'].append(self.round_sig(val_loss, 4))
        self.history['val_profit'].append(self.round_sig(val_profit, 4))
        self.history['learning_rate'].append(learning_rate)
        self.history['recall'].append(self.round_sig(recall, 4))
        self.history['precision'].append(self.round_sig(precision, 4))
        self.history['time_taken'].append(time_taken)
        self.history['gauge'].append(gauge)
        self.history['backtest_reports'].append(backtest_report)
        self.save()

    def get_zipped_history(self):
        z = zip(self.history['loss'], self.history['val_loss_0'], self.history['val_profit_0'], self.history['val_loss'], self.history['val_profit'], self.history['learning_rate'], self.history['recall'], self.history['precision'], self.history['time_taken'])
        return list(z)
    # def append_backtests(self, epoch, key_a, key_b, interval_profit, backtest):
    #     self.history['backtests'].append((epoch, key_a, key_b, interval_profit, backtest))
    #     self.save()
    def get_backtest_report(self, epoch):
        return self.history['backtest_reports'][epoch]     # sure exists. epoch is selected.
    def len(self):
        assert len(self.history['loss']) == len(self.history['val_loss_0'])
        assert len(self.history['loss']) == len(self.history['val_profit_0'])
        assert len(self.history['loss']) == len(self.history['val_loss'])
        assert len(self.history['loss']) == len(self.history['val_profit'])
        assert len(self.history['loss']) == len(self.history['recall'])
        assert len(self.history['loss']) == len(self.history['precision'])
        assert len(self.history['loss']) == len(self.history['learning_rate'])
        assert len(self.history['loss']) == len(self.history['time_taken'])
        assert len(self.history['loss']) == len(self.history['gauge'])
        assert len(self.history['loss']) == len(self.history['backtest_reports'])
        return len(self.history['loss'])
    def get_min_val_loss(self):
        return float('inf') if self.len() <= 0 else min(self.history['val_loss'])
    def get_max_gauge(self, epoch):
        gauges = self.history['gauge']
        return -float('inf') if (len(gauges) <= 0 or epoch <= 0) else max(gauges[:epoch])
    def replace_gauge(self, epoch, gauge):
        self.history['gauge'][epoch] = gauge;   self.save()
    def show(self, ax, show_val_0=True):
        ax.set_title(MODEL_ID + ": training history: max gauge = " + str(max(self.history['gauge']) if len(self.history['gauge']) > 0 else 0))

        ax.plot(self.history['loss'], label='train_loss')
        if show_val_0: ax.plot(self.history['val_loss_0'], label='GET_VAL_LOSS_0')
        ax.plot(self.history['val_loss'], label='val_loss')
        ax.plot([-v for v in self.history['val_loss']], label='vector_profit')
        ax.plot(self.history['val_profit'], label='scalar_profit')
        
        ax.legend(loc='lower left')
        ax.grid(True)
        ax.set_ylabel('loss or profit')
        ax.set_xlabel('epoch', loc='right')

In [ ]:
class recall():
  def __init__(self, **kwargs):
    self.n = None
    self.recall = None
    self.reset()

  def update(self, label, pred):    # (batch,)
    label = tf.cast(label, dtype=tf.float32)
    pred = tf.cast(pred, dtype=tf.float32)
    hit_positives = tf.math.reduce_sum(label * pred, axis=None)
    labeled_positives = tf.math.reduce_sum(label, axis=None)
    recall = hit_positives / (labeled_positives + 1e-9) #tf.keras.backend.epsilon())
    self.n += 1
    self.recall = self.recall * (self.n-1)/self.n + recall / self.n

  def result(self):
    return self.recall
  
  def reset(self):
    self.n = 0
    self.recall = tf.Variable(0.0, dtype=tf.float32, trainable=False)
  
recall_object = recall(min_seq_len=5)

class precision():
  def __init__(self, **kwargs):
    self.n = None
    self.precision = None
    self.reset()

  def update(self, label, pred):
    label = tf.cast(label, dtype=tf.float32)
    pred = tf.cast(pred, dtype=tf.float32)
    hit_positives = tf.math.reduce_sum(label * pred, axis=None)
    predicted_positives = tf.math.reduce_sum(pred, axis=None)
    precision = hit_positives / (predicted_positives + 1e-9) #tf.keras.backend.epsilon())
    self.n += 1
    self.precision = self.precision * (self.n-1)/self.n + precision / self.n

  def result(self):
    return self.precision
  
  def reset(self):
    self.n = 0
    self.precision = tf.Variable(0.0, dtype=tf.float32, trainable=False)

precision_object = precision(min_seq_len=5)

In [ ]:
@tf.function    # Removing this decoration leads to GPU OOM!!!
def train_step(model, optimizer, x, y):
    
    with tf.GradientTape() as tape:
        outputs = model(x, training=True)  # (nBookies, batch, nQueries)
        loss_sum, _, _ = model.loss(y, outputs)    # (), (), (batch,)

    grads = tape.gradient(loss_sum, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    del tape    # new
    return loss_sum  # (), (batch,)

@tf.function
def find_loss_for_dataset_step(model, x, y):
    outputs = model(x, training=False)  # (nBookies, batch, nQueries)
    loss_sum, profit_sum, profits = model.loss(y, outputs)    # (), (), (batch,)
    return loss_sum, profit_sum, profits #, profits

def find_loss_for_dataset(model, ds_batches):
    prev_nSum = new_nSum = 0
    avg_loss = tf.Variable(0.0, dtype=tf.float32, trainable=False) 
    avg_profit = tf.Variable(0.0, dtype=tf.float32, trainable=False)
    total_profits = tf.Variable([0.0], dtype=tf.float32, trainable=False)
    for step, ((baseId, sequence, base_bb, baseDateDetails, mask), y) in enumerate(ds_batches):
        (base_label, seq_type) = y
        x = (sequence, base_bb, baseDateDetails, mask)
        loss_sum, profit_sum, profits = find_loss_for_dataset_step(model, x, y)
        batch_size = baseId.shape[0]
        new_nSum += batch_size  # += baseId.shape[0], given sum of loss and profit. Ignore that the last batch might be smaller than others.
        avg_loss = avg_loss * prev_nSum / new_nSum + loss_sum / new_nSum
        avg_profit = avg_profit * prev_nSum / new_nSum + profit_sum / new_nSum
        prev_nSum = new_nSum
        total_profits = tf.concat([total_profits, profits], axis=-1)
    return avg_loss, avg_profit, total_profits    # agerage loss and backtest_profit per game

@tf.function
def find_output_for_dataset_step(model, x):
    outputs = model(x, training=False)  # (nBookies, batch, nQueries)
    return outputs

def find_output_for_dataset(model, ds_batches):
    prediction = {}

    for step, ((baseId, sequence, base_bb, baseDateDetails, mask), _) in enumerate(ds_batches):
        x = (sequence, base_bb, baseDateDetails, mask)
        stake_p = find_output_for_dataset_step(model, x)    # (nBookies, batch, nQueries)
        one_hot_stake_p = tf.squeeze(tf.one_hot(tf.nn.top_k(stake_p).indices, tf.shape(stake_p)[-1]), axis=-2)   # one_hot stake_p, (nBookies, batch, nQueries)
        one_hot_stake_p = tf.transpose(one_hot_stake_p, perm=[1, 0, 2])   # (batch, nBookies, nQueries)
        one_hot_stake_p = tf.reshape(one_hot_stake_p, [one_hot_stake_p.shape[0], -1])   # (batch, nBookies * nQueries)

        baseId = baseId.numpy()     # (batch,)
        one_hot_stake_p = one_hot_stake_p.numpy()   # (batch, nBookies * nQueries)
        prediction = prediction | { baseId[b] : list(one_hot_stake_p[b].astype(np.int32)) for b in range(baseId.shape[0]) }
        # output = tf.reshape(output, [output.shape[0], self.nQueries, -1])    # (batch, nQueries, nBookies)
        # output = tf.transpose(output, perm=[2, 0, 1])     # (nBookies, batch, nQueries)       
    return prediction

class Adam_exponential(tf.keras.optimizers.Adam):
    def __init__(self, initial_step, starting_rate, ex_step, ex_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-12):
        self.step = tf.Variable(initial_step, dtype=tf.float32, trainable=False)
        learning_rate = tf.compat.v1.train.exponential_decay(starting_rate, self.step, ex_step, ex_rate/starting_rate, staircase=False)
        super().__init__(learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)

# @tf.function
def find_recall_precision(model, ds_batches):
    recall_object.reset(); precision_object.reset()
    for step, ((baseId, sequence, base_bb, baseDateDetails, mask), (base_label, seq_type)) in enumerate(ds_batches):
        x = (sequence, base_bb, baseDateDetails, mask)
        pred = model(x, training=False)
        pred = tf.cast(pred > 0.5, dtype=tf.int32)
        # Wrong. transorm base_label to label. It has odds too.   #-------------------------------------------------------- Wrong
        recall_object.update(base_label, pred); precision_object.update(base_label, pred)
    return recall_object.result(), precision_object.result()


In [ ]:
def show_sub_epoch(epoch, batch_id_seen, sub_loss, epoch_loss, samples_seen, learning_rate):
    print("epoch: {}, batchId_seen: {}, sub_loss: {:.6}, epoch_loss: {:.6f}, samples_seen: {}, learning_rate: {:.5e}                  ".
            format(epoch, batch_id_seen, float(sub_loss), float(epoch_loss), samples_seen, learning_rate), end='\r')

# conclude_train_epoch(-1, 0, 0, 0, 0, playback=False)
def conclude_train_epoch(epoch, model, history, checkpointPath, checkpointPathBest, train_loss, val_loss_0, val_profit_0, val_loss, val_profit, learning_rate, recall, precision, time_taken, playback=False, gaugeTerm='val_loss'):
    global focus_interval_id, focus_scores, focus_back, focus_valid, focus_test
    print("epoch: {}, train_loss: {:.5f}, val_loss_0: {:.5f}, val_profit_0: {:.5f}, val_loss: {:.5f}, val_profit: {:.5f}, learning_rate: {:.5e}, recall/precision: {:.4f}/{:.4f}, time_taken: {:.1f} m".format(epoch, train_loss, val_loss_0, val_profit_0, val_loss, val_profit, learning_rate, recall, precision, time_taken))

    backtest_report = ()
    if gaugeTerm == 'val_loss': gauge = - val_loss
    elif gaugeTerm == 'recall': gauge = recall
    elif gaugeTerm == 'precision': gauge = precision
    elif gaugeTerm == 'val_profit': gauge = val_profit

    # backtest_report, gauge = run_backtest(epoch, model, history, playback)   #-------------------------------------------------

    upgraded = False
    #-------------------- get interval scores from arguments, create the best checkpoint if we have a highest ever score.
    if gauge > history.get_max_gauge(epoch):
        upgraded = True
        # focus_interval_id, focus_scores, focus_back, focus_valid, focus_test = bets_interval_id, interval_scores, best_interval_back, best_interval_valid, best_interval_test
        if not playback: model.save_weights(checkpointPathBest)
        print("---------")
    if playback: history.replace_gauge(epoch, gauge)

    #--------------------- Save, finally
    if not playback:
        model.save_weights(checkpointPath)
        # self, train_loss, val_loss, learning_rate, recall, precision, time_taken, gauge, backtest_report
        history.append(train_loss, val_loss_0, val_profit_0, val_loss, val_profit, learning_rate, recall, precision, time_taken, gauge, backtest_report)

    return upgraded

In [ ]:
checkpointPath = os.path.join(checkpoint_folder_path, MODEL_ID + '_weights')
checkpointPathBest = os.path.join(checkpoint_folder_path, MODEL_ID + '_weights_best')
historyPath = os.path.join(checkpoint_folder_path, MODEL_ID + '_history.json')
history = history_class(historyPath); history.load()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 5))
history.show(ax, show_val_0=GET_VAL_LOSS_0)

In [ ]:
model_1x2 = create_model_object(Model_1X2)

In [ ]:
#-------------------------------------------------------------------------------------------------
try:    model_1x2.load_weights(checkpointPath)
except: pass
epoch = 0   # no gauge, as it's not an argument but measured in conclude_train_epoch.
for train_loss, val_loss_0, val_profit_0, val_loss, val_profit, learning_rate, recall, precision, time_taken in history.get_zipped_history():
    _ = conclude_train_epoch(epoch, model_1x2, history, checkpointPath, checkpointPathBest, train_loss, val_loss_0, val_profit_0, val_loss, val_profit, learning_rate, recall, precision, time_taken, playback = True, gaugeTerm='val_profit')
    epoch += 1

In [ ]:
# @tf.function  #-------------------- Weird: no work.
def train(epochs, nUpgrades, model, optimizer, history, checkpointPath, checkpointPathBest, train_ds, valid_batches, apply_train_pipeline, get_f1=False, gaugeTerm='val_loss', get_val_loss_0=False):
    epochs = epochs; seen_upgrades = 0
    for epoch in range(history.len(), history.len() + epochs):
        start_time = time.time()
        optimizer.step.assign(history.len()); learning_rate = optimizer.lr.numpy()
        prev_nSum = new_nSum = 0; prev_mSum = new_mSum = 0; epoch_loss = 0.; sub_loss = 0.
        samples_seen = 0 # sub_loss = tf.Variable(0.0, dtype=tf.float32, trainable=False)
        set_seed(epoch)
        train_batches = apply_train_pipeline(train_ds)      # This is REQUIRED to reshuffle the data at each iteration. Tested.
        for batch_id, ((baseId, sequence, base_bb, baseDateDetails, mask), y) in enumerate(train_batches):    # reshuffle at each iteration - NO!
            x = (sequence, base_bb, baseDateDetails, mask)
            samples_seen += sequence.shape[0]
            loss_sum = train_step(model, optimizer, x, y)  # ()

            batch_size = baseId.shape[0]
            new_nSum += batch_size; new_mSum += batch_size
            sub_loss = sub_loss * prev_nSum / new_nSum + loss_sum / new_nSum
            epoch_loss = epoch_loss * prev_mSum / new_mSum + loss_sum / new_mSum
            prev_nSum = new_nSum; prev_mSum = new_mSum

            if batch_id % int(100/BATCH_SIZE) == 0:
                # optimizer.step.assign(history.len()); learning_rate = optimizer.lr.numpy()  #----------------------------------------------------
                show_sub_epoch(epoch, batch_id, sub_loss, epoch_loss, samples_seen, learning_rate); prev_nSum = new_nSum = 0; sub_loss = 0.0

        show_sub_epoch(epoch, batch_id, sub_loss, epoch_loss, samples_seen, learning_rate)  # closing show
        val_loss_0 = val_profit_0 = 0.0
        if get_val_loss_0: val_loss_0, val_profit_0, _ = find_loss_for_dataset(model, train_batches)
        val_loss, val_profit, _ = find_loss_for_dataset(model, valid_batches)

        recall = precision = -1.0
        if get_f1: recall, precision = find_recall_precision(model, valid_batches); recall = recall.numpy(); precision = precision.numpy()
        
        # epoch, model, history, checkpointPath, checkpointPathBest, epoch_loss, val_loss, learning_rate, recall, precision, time_taken, playback=False
        upgraded = conclude_train_epoch(epoch, model, history, checkpointPath, checkpointPathBest, float(epoch_loss), float(val_loss_0), float(val_profit_0), float(val_loss), float(val_profit), learning_rate, recall, precision, (time.time()-start_time)/60, playback=False, gaugeTerm=gaugeTerm)
        if upgraded: seen_upgrades += 1
        if seen_upgrades >= nUpgrades: break

    return seen_upgrades


In [ ]:
if TRAIN_MODE == True and OPERATION == 'PRETRAIN': # model_1x2 is trained to find profitable staking vectors for games, although average profitability is far below zero.
    try:    model_1x2.load_weights(checkpointPath).expect_partial(); print("model_1x2 loaded its previous checkpoint.")
    except: print("model_1x2 loaded its initial weights.")    # The raw model_1x2 itself is the starting point.
    optimizer = Adam_exponential(history.len(), STARTING_LEARNING_RATE, EXAMPLE_LEARNING_STEP, EXAMPLE_LEARNING_RATE, beta_1=0.9, beta_2=0.99, epsilon=1e-12)
    # Goal: get checkpointPathBest checkpoint, which is used as the starting checkpoint of both TRAIN_C and FINETUNE.
    nEpochs = 300; wanted_upgrades = 60
    # epochs, nUpgrades, model, optimizer, history, checkpointPath, checkpointPathBest, train_ds, valid_batches, apply_train_pipeline, get_f1=False
    nUpgrades = train(nEpochs, wanted_upgrades, model_1x2, optimizer, history, checkpointPath, checkpointPathBest, train_ds, valid_batches, apply_train_pipeline, get_f1 = False, gaugeTerm='val_profit', get_val_loss_0=GET_VAL_LOSS_0)
    
    OPERATION = 'TRAIN_C'   # Certificate for the next operation

In [ ]:
if OPERATION == 'BACKTEST':
    startDate, endDate = YK_assistant.find_first_start_and_end_date(valid_ds)
    valid_days = (endDate-startDate).days + 1
    model_1x2.load_weights(checkpointPathBest)
    avg_loss, avg_profit, profits = find_loss_for_dataset(model_1x2, valid_batches)
    profits = list(profits.numpy())

In [ ]:
if OPERATION == 'BACKTEST':
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 5))
    plt.plot(profits); plt.ylim([-1.5, 5])
    plt.grid(True)
    plt.title("Model {}: Profit Per Game, {} major 4 football leagues, spaning {} days, betting 1.0 on each game, Agerage: {:.4f}"
              .format(MODEL_ID, COUNTRY, valid_days, sum(profits)/len(profits)))
    plt.xlabel('game'); plt.ylabel('profit')
    plt.show()

    balance_history, max_balance, linear_deepest_canyon = YK_assistant.linear_strategy(profits)
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 5))
    plt.title("Model {}: Accumulated Profits, {} major 4 football leagues, spaning {} days, betting 1.0 on each game. DeepestCanyon: {}"
              .format(MODEL_ID, COUNTRY, valid_days, int(linear_deepest_canyon)+1))
    plt.plot(balance_history)
    plt.grid(True)
    plt.xlabel('game'); plt.ylabel('accumulated profit')
    plt.show()

In [ ]:
if OPERATION == 'BACKTEST':
    safety = (int(linear_deepest_canyon) + 1) * 2
    balance_history, max_balance, deepest_canyon = YK_assistant.exponential_strategy(profits, safety)
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 5))
    plt.title("Model {}: Accumulated Profits, {} major 4 football leagues, spaning {} days, betting a {}-th on each game. DeepestCanyon: {}"
            .format(MODEL_ID, COUNTRY, valid_days, safety, int(deepest_canyon)+1))
    plt.plot(balance_history)
    plt.grid(True)
    plt.xlabel('game'); plt.ylabel('accumulated profit')
    plt.show()

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 5))
    plt.title("Zooming into the first 500 games")
    plt.plot(balance_history[:500])
    plt.grid(True)
    plt.xlabel('game'); plt.ylabel('accumulated profit')
    plt.show()


In [ ]:
next_or_same_sheet = 'same'     # 'next', 'same'

if not TRAIN_MODE:      # just sketch
    bookie_dict = YK_assistant.find_latest_bookie_dictionary(countryTheme_folder_path, DIVIISONS)
    bookie_list = list(bookie_dict.keys())
    
    filename = MODEL_ID + '.xlsx'; sheetname = next_or_same_sheet
    input_columns, sheetName = YK_assistant.format_inference_input_excel_sheet(history_folder, filename, sheetname, bookie_list)
    print(input_columns)
    print(sheetName)

In [ ]:
if not TRAIN_MODE:      # just sketch
    consistent, check_report = YK_assistant.check_for_consistency_input_excel_sheet(countryTheme_folder_path, history_folder, filename, sheetName, input_columns, tokenizer_team, bookie_list, DIVIISONS)
    print('Consistent: ', consistent)
    print(check_report)

In [ ]:

import pandas as pd

if not TRAIN_MODE: # and consistent:
    df_bbab_to_predict = YK_assistant.build_bbab_dataframe(history_folder, filename, sheetName, BB_cols, AB_cols, bookie_dict)

    YK_assistant.assign_seasonal_filenames(gameHistory_folder_path)
    df_train, df_bbab_not_in_train = YK_assistant.CREATE_DADAFRAMES_v2(gameHistory_folder_path, countryTheme_folder_path, Required_Non_Odds_cols, NUMBER_BOOKIES, oddsGroupsToExclude = BOOKIE_TO_EXCLUDE, preferedOrder = PREFERED_ORDER, train_mode = TRAIN_MODE, skip=True)
    print("df_train: ", df_train.shape)
    print("df_bbab_not_in_train: ", df_bbab_not_in_train.shape)

    df_sequence = pd.concat([df_train, df_bbab_not_in_train])   # No duplicates. Maximized bbab dataframe.
    df_base = df_bbab_to_predict                                  # Minimized
    # Do NOT save, because the 'id' column for df_bb_to_predict is temporary and arbitrary.
    predict_map = YK_assistant.CREATE_MAP_v2(map_folder_path, map_filename_base, HISTORY_LEN, df_sequence, df_base, year_span=SEQ_YEAR_SPAN, testcount=-1, to_save=False)
    print('predict_map', predict_map)

    df_sequence = pd.concat([df_train, df_bbab_not_in_train])   # No duplicates. Maximized bbab dataframe.
    df_base = df_bbab_to_predict; existing_ds = None; total_map = predict_map                                     # Minimized
    # Let the function use a dummy Y, as df_base has not Y's.
    predict_ds = CREATE_DATASET(df_sequence, df_base, existing_ds, total_map, tokenizer_team, normalization_parms)  # defines, not makes, predict_ds
    predict_batches = apply_test_pipeline(predict_ds)

    model_1x2.load_weights(checkpointPathBest)
    prediction = find_output_for_dataset(model_1x2, predict_batches)

    YK_assistant.create_output_excel_sheet(history_folder, filename, sheetName, prediction, df_bbab_to_predict, bookie_dict, AB_cols, NUMBER_QUERIES)